# FAIRE 데이터셋 기반 LLM 채용 편향성 분석

이 노트북은 **FAIRE (Fairness Assessment in Resume Evaluation)** 데이터셋을 사용하여, LLM 기반 이력서 평가 모델의 편향성을 심층적으로 분석하는 과정을 담고 있습니다.

FairFace 분석 노트북의 구조를 템플릿으로 삼아, 텍스트 데이터(NLP) 환경에 맞게 데이터 처리, 모델, 학습 방식을 수정하였습니다.

**실험 목표:**
1.  **Baseline 모델 구축**: 이력서 텍스트로 사전 학습된 언어 모델(RoBERTa)을 학습하고, 이력서 평가 점수 예측에 대한 기본적인 성능과 그룹별 공정성 지표를 측정합니다.
2.  **Micro-Bias Sensitivity Curve 분석**: 데이터셋의 인종/성별 그룹 비율을 미세하게 조정했을 때, 모델의 평가 결과가 얼마나 민감하게 변하는지 측정합니다.
3.  **Over-Correction Damage Index (ODI) 계산**: 편향 완화 기법(Reweighing) 적용 시 발생하는 성능 저하(Trade-off)를 정량적으로 분석합니다.
4.  **Hidden Subgroup Discovery**: 특정 인종과 성별이 조합된 하위집단에서 발생하는 잠재적 편향을 탐지합니다.

---

## 단계 1: 실험 환경 설정 및 데이터셋 준비

텍스트 처리를 위해 `transformers` 라이브러리를 추가하고, FAIRE 데이터셋을 불러올 준비를 합니다. 텍스트를 모델이 학습할 수 있는 형태(Tokenized Tensors)로 변환하는 PyTorch `Dataset`과 `DataLoader`를 정의합니다.


In [ ]:
# === 1.1 라이브러리 임포트 ===
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import copy
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Sampler
from collections import defaultdict

# NLP 처리를 위한 transformers 라이브러리
# TODO: 'pip install transformers' 실행 필요
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup

# 경고 메시지 무시
import warnings
warnings.filterwarnings('ignore')

# Matplotlib 스타일 설정
plt.style.use('seaborn-v0_8-whitegrid')

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")


# === 1.2 주요 설정 (Configuration) ===
# TODO: README에 따라 FAIRE 데이터셋을 'data/FAIRE/' 경로에 준비해주세요.
DATA_DIR = './data/FAIRE/'
# FAIRE 데이터셋은 보통 하나의 파일로 제공됩니다. 실제 파일명에 맞게 수정해야 할 수 있습니다.
# 예시로 'faire_dataset.csv'를 사용합니다.
DATA_FILE = os.path.join(DATA_DIR, 'all_ideology_paired.csv') # FAIRE 저장소의 실제 파일명 예시

# 실험 속성
# FAIRE 데이터셋의 실제 컬럼명에 맞게 수정이 필요할 수 있습니다.
TARGET_ATTR = 'score'  # 예측 대상: 적합성 점수 (보통 1~5점 척도)
SENSITIVE_ATTR = 'race' # 민감 속성 ('race' 또는 'gender')

# 모델 및 학습 하이퍼파라미터
MODEL_NAME = 'roberta-base' # 사용할 사전 학습 모델
MAX_LEN = 256 # 토크나이저의 최대 입력 길이
BATCH_SIZE = 16 # GPU 메모리에 맞춰 조정
NUM_WORKERS = 0
NUM_EPOCHS = 4
LEARNING_RATE = 2e-5
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 파일 저장 경로
MODEL_SAVE_PATH = 'best_faire_model.pth'
RESULTS_DIR = 'results_faire'
os.makedirs(RESULTS_DIR, exist_ok=True)

print(f"Device: {DEVICE}")
print(f"Target Attribute: {TARGET_ATTR}")
print(f"Sensitive Attribute: {SENSITIVE_ATTR}")
print(f"Using Model: {MODEL_NAME}")


In [ ]:
# === 1.3 FAIRE 데이터셋 클래스 정의 ===

class FAIREDataset(Dataset):
    """FAIRE 이력서 텍스트 데이터셋을 위한 PyTorch Dataset 클래스"""
    
    def __init__(self, filepath, tokenizer, max_len, text_column='text', target_column='score'):
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.text_column = text_column
        self.target_column = target_column
        
        try:
            self.df = pd.read_csv(filepath)
            # 결측치가 있는 행 제거
            self.df = self.df.dropna(subset=[self.text_column, self.target_column, 'gender', 'race']).reset_index(drop=True)
        except FileNotFoundError:
            print(f"Error: Data file not found at {filepath}")
            print("Please check the 'DATA_FILE' variable and ensure the dataset is prepared correctly.")
            self.df = pd.DataFrame()
            return

        # 라벨 인코딩
        # 점수(score)는 회귀(regression) 문제로 다룰 수도 있지만, 여기서는 분류(classification) 문제로 접근하기 위해 정수로 변환합니다.
        # FAIRE 점수는 1-5점이므로, 0-4점으로 매핑합니다.
        self.df[self.target_column] = self.df[self.target_column].astype(int) - 1
        
        self.gender_map = {label: i for i, label in enumerate(sorted(self.df['gender'].unique()))}
        self.race_map = {label: i for i, label in enumerate(sorted(self.df['race'].unique()))}
        self.idx_to_gender = {v: k for k, v in self.gender_map.items()}
        self.idx_to_race = {v: k for k, v in self.race_map.items()}

        print(f"Dataset loaded from {filepath}. Number of samples: {len(self.df)}")
        print(f"Target classes: {self.df[self.target_column].unique()}")

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = str(self.df.loc[idx, self.text_column])
        score = self.df.loc[idx, self.target_column]
        gender = self.gender_map[self.df.loc[idx, 'gender']]
        race = self.race_map[self.df.loc[idx, 'race']]

        # 텍스트 토크나이징
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'targets': torch.tensor(score, dtype=torch.long),
            'gender': torch.tensor(gender, dtype=torch.long),
            'race': torch.tensor(race, dtype=torch.long),
        }

# --- 토크나이저 초기화 ---
# 사전 학습된 모델에 맞는 토크나이저를 불러옵니다.
tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)
print("Tokenizer initialized.")


In [ ]:
# === 1.4 데이터셋 분할 및 데이터로더 생성 ===
from sklearn.model_selection import train_test_split

# FAIRE 데이터셋은 train/val이 분리되어 있지 않으므로, 직접 분할합니다.
try:
    # 전체 데이터셋 로드
    full_dataset = FAIREDataset(
        filepath=DATA_file,
        tokenizer=tokenizer,
        max_len=MAX_LEN,
        target_column=TARGET_ATTR
    )

    if not full_dataset.df.empty:
        # train_test_split을 사용하여 인덱스를 분할 (stratify를 사용하여 타겟 분포 유지)
        train_indices, val_indices = train_test_split(
            range(len(full_dataset)),
            test_size=0.2,
            random_state=42,
            stratify=full_dataset.df[TARGET_ATTR]
        )

        # Subset을 사용하여 데이터셋 분할
        train_dataset = torch.utils.data.Subset(full_dataset, train_indices)
        val_dataset = torch.utils.data.Subset(full_dataset, val_indices)

        # 데이터로더 생성
        dataloaders = {
            'train': DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS),
            'val': DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
        }
        
        dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}
        
        print(f"\nDataset split into train and validation sets.")
        print(f"Train dataset size: {dataset_sizes['train']}")
        print(f"Validation dataset size: {dataset_sizes['val']}")

        # 라벨 맵 정보 저장 (나중에 사용)
        label_maps = {
            'gender': full_dataset.gender_map,
            'race': full_dataset.race_map
        }

    else:
        dataloaders = None
        print("Dataset is empty. Cannot create dataloaders.")

except Exception as e:
    print(f"An error occurred: {e}")
    print("Please check the file path and column names in the configuration cell.")
    dataloaders = None



---
## 단계 2: Baseline 모델 구축 및 학습

이제 텍스트 데이터 준비가 완료되었으니, 사전 학습된 언어 모델(RoBERTa)을 기반으로 Baseline 모델을 구축하고 FAIRE 데이터셋으로 fine-tuning을 진행합니다.

**수행 작업:**
1.  **모델 정의**: HuggingFace `transformers` 라이브러리를 사용해 `roberta-base` 모델을 불러온 후, 우리의 예측 목표(점수 분류, 5개 클래스)에 맞게 출력 레이어를 수정합니다.
2.  **학습 함수 정의**: 텍스트 데이터를 입력받아 모델을 학습하고 검증하는 `train_model` 함수를 정의합니다. Transformers 모델 학습에 표준적으로 사용되는 `AdamW` 옵티마이저와 `linear schedule with warmup`을 적용합니다.
3.  **모델 학습 실행**: 정의된 함수와 데이터로더를 사용하여 실제로 모델 학습을 시작합니다. 학습 과정에서의 손실(loss)과 정확도(accuracy)가 출력됩니다.


In [ ]:
# === 2.1 모델 정의 및 학습/평가 함수 ===

def get_faire_model(num_classes):
    """
    사전 학습된 RoBERTa 모델을 로드하고 FAIRE 데이터셋의 점수 예측을 위해
    마지막 레이어를 수정합니다.
    """
    model = RobertaForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=num_classes,
        output_attentions=False,
        output_hidden_states=False
    )
    return model.to(DEVICE)


def train_model(model, dataloaders, optimizer, scheduler, num_epochs=4):
    """NLP 모델 학습 및 검증을 위한 메인 루프"""
    since = time.time()
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for batch in tqdm(dataloaders[phase], desc=f"{phase.capitalize()} Phase"):
                input_ids = batch['input_ids'].to(DEVICE)
                attention_mask = batch['attention_mask'].to(DEVICE)
                targets = batch['targets'].to(DEVICE)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(
                        input_ids=input_ids,
                        attention_mask=attention_mask,
                        labels=targets
                    )
                    loss = outputs.loss
                    preds = torch.argmax(outputs.logits, dim=1)

                    if phase == 'train':
                        loss.backward()
                        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # Gradient clipping
                        optimizer.step()
                        scheduler.step()

                running_loss += loss.item() * input_ids.size(0)
                running_corrects += torch.sum(preds == targets)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
            history[f'{phase}_loss'].append(epoch_loss)
            history[f'{phase}_acc'].append(epoch_acc.item())

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), MODEL_SAVE_PATH)
                print(f"** Best validation accuracy updated: {best_acc:.4f}, Model saved to {MODEL_SAVE_PATH}")

    time_elapsed = time.time() - since
    print(f'\nTraining complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    model.load_state_dict(best_model_wts)
    return model, history

def plot_training_history(history):
    """학습 과정의 손실과 정확도 변화를 시각화합니다."""
    # (FairFace 노트북과 동일한 함수 사용)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    ax1.plot(history['train_loss'], label='Train Loss')
    ax1.plot(history['val_loss'], label='Validation Loss')
    ax1.set_title('Model Loss')
    ax1.set_xlabel('Epochs'); ax1.set_ylabel('Loss'); ax1.legend()
    ax2.plot(history['train_acc'], label='Train Accuracy')
    ax2.plot(history['val_acc'], label='Validation Accuracy')
    ax2.set_title('Model Accuracy')
    ax2.set_xlabel('Epochs'); ax2.set_ylabel('Accuracy'); ax2.legend()
    fig.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, 'faire_baseline_training_history.png'))
    plt.show()

print("Model and helper functions are defined for FAIRE dataset.")
print("You can now run the next cell to start training.")


In [ ]:
# === 2.2 Baseline 모델 학습 실행 ===

if dataloaders:
    # FAIRE 데이터셋의 타겟은 1~5점이므로 5개 클래스
    NUM_CLASSES = 5 
    baseline_model = get_faire_model(num_classes=NUM_CLASSES)
    
    # 옵티마이저 설정
    optimizer = AdamW(baseline_model.parameters(), lr=LEARNING_RATE, correct_bias=False)
    
    # 스케줄러 설정
    total_steps = len(dataloaders['train']) * NUM_EPOCHS
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )

    # 모델 학습 시작
    baseline_model, history = train_model(
        baseline_model,
        dataloaders,
        optimizer,
        scheduler,
        num_epochs=NUM_EPOCHS
    )
    
    # 학습 결과 시각화
    print("\nPlotting training history...")
    plot_training_history(history)
    
else:
    print("Dataloaders are not available. Cannot start training.")
    print("Please go back and ensure the dataset is loaded correctly.")


---
### 단계 2.3: Baseline 모델 성능 및 공정성 평가

FAIRE 데이터셋에 대한 모델 학습이 완료되었습니다. 이제 검증 데이터셋(validation set)을 사용하여 학습된 모델의 전반적인 이력서 평가 성능과 그룹별 공정성 지표를 평가합니다.


In [ ]:
# === 2.3 성능 및 공정성 평가 함수 정의 및 실행 ===

def evaluate_faire_fairness(model, dataloader):
    """FAIRE 모델의 성능과 공정성 지표를 상세히 평가하고 결과를 반환합니다."""
    model.eval()
    
    y_true, y_pred, sensitive_attrs = [], [], []
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(DEVICE)
            attention_mask = batch['attention_mask'].to(DEVICE)
            targets = batch['targets'].to(DEVICE)
            s_attrs = batch[SENSITIVE_ATTR].to(DEVICE)
            
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            preds = torch.argmax(outputs.logits, dim=1)
            
            y_true.extend(targets.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            sensitive_attrs.extend(s_attrs.cpu().numpy())

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    sensitive_attrs = np.array(sensitive_attrs)
    
    results = {}
    results['overall_acc'] = np.mean(y_true == y_pred)
    
    group_accuracies, group_sizes = {}, {}
    # full_dataset을 참조하여 인덱스-이름 맵 가져오기
    original_dataset = dataloader.dataset.dataset 
    s_map = original_dataset.idx_to_race if SENSITIVE_ATTR == 'race' else original_dataset.idx_to_gender
    
    for group_idx, group_name in s_map.items():
        mask = (sensitive_attrs == group_idx)
        if mask.sum() > 0:
            group_accuracies[group_name] = np.mean(y_true[mask] == y_pred[mask])
            group_sizes[group_name] = mask.sum()
    
    results['group_accuracies'] = group_accuracies
    results['group_sizes'] = group_sizes
    
    if group_accuracies:
        results['accuracy_gap'] = max(group_accuracies.values()) - min(group_accuracies.values())
    
    return results

# FairFace의 시각화 함수 재사용 (파일 저장 경로만 수정)
def plot_faire_fairness_results(results):
    group_accuracies = results['group_accuracies']
    group_names = list(group_accuracies.keys())
    acc_values = list(group_accuracies.values())
    
    plt.figure(figsize=(12, 6))
    bars = plt.bar(group_names, acc_values, color=sns.color_palette('viridis', len(group_names)))
    plt.title('FAIRE Baseline Model Accuracy by Sensitive Group', fontsize=16)
    plt.xlabel('Sensitive Attribute Group'); plt.ylabel('Accuracy'); plt.ylim(0, 1.0)
    plt.xticks(rotation=45, ha='right')
    
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2.0, yval, f'{yval:.3f}', va='bottom', ha='center')
        
    plt.axhline(y=results['overall_acc'], color='r', linestyle='--', label=f"Overall Acc: {results['overall_acc']:.3f}")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, 'faire_baseline_fairness_evaluation.png'))
    plt.show()

# === 평가 실행 ===
if 'baseline_model' in locals():
    model_to_evaluate = get_faire_model(num_classes=5)
    try:
        model_to_evaluate.load_state_dict(torch.load(MODEL_SAVE_PATH))
        print(f"Model loaded from {MODEL_SAVE_PATH} for evaluation.")
        
        baseline_results = evaluate_faire_fairness(model_to_evaluate, dataloaders['val'])

        print("\n--- FAIRE Baseline Model Evaluation Results ---")
        print(f"Overall Accuracy: {baseline_results['overall_acc']:.4f}")
        print("\nGroup Accuracies:")
        for group, acc in baseline_results['group_accuracies'].items():
            print(f"  - {group}: {acc:.4f} (n={baseline_results['group_sizes'][group]})")
        print(f"\nMaximum Accuracy Gap: {baseline_results.get('accuracy_gap', 0):.4f}")
        
        plot_faire_fairness_results(baseline_results)
    except FileNotFoundError:
        print(f"Error: Saved model not found. Please run the training cell first.")
else:
    print("Baseline model not available. Cannot run evaluation.")


---
## 단계 3: Micro-Bias Sensitivity Curve (편향 민감도 곡선) 분석

FAIRE Baseline 모델의 성능 평가를 마쳤습니다. 이제 이 모델이 데이터셋에 존재하는 미세한 인구 통계학적 편향 변화에 얼마나 민감하게 반응하는지를 측정합니다.

**실험 목표:**
"이력서 데이터셋에서 특정 그룹(예: 'White')의 비율이 아주 약간만 변해도, 모델의 채용 점수 예측이 해당 그룹에 유리하거나 불리하게 바뀌는가?"라는 질문에 답하는 것입니다. 

**수행 작업:**
1.  **편향 주입 샘플러(`BiasedSampler`) 정의**: 검증 데이터셋에서 특정 인종 그룹의 샘플 비율을 정밀하게 조절하여 데이터를 추출하는 커스텀 `Sampler`를 정의합니다.
2.  **민감도 분석 실행**: 편향 비율을 단계적으로 변화시키면서, 각 단계마다 편향이 주입된 데이터로 모델을 평가합니다. 전체 정확도와 그룹별 정확도 차이의 변화를 기록합니다.
3.  **결과 시각화**: X축을 '주입된 편향의 강도', Y축을 '성능 및 공정성 지표'로 하는 "편향 민감도 곡선"을 그려, 변화의 추이를 직관적으로 분석합니다.


In [ ]:
# === 3.1 편향 주입 샘플러 및 민감도 분석/시각화 ===

# --- 실험 설정 ---
BIAS_INJECTION_GROUP = 'White' 
# 'White' 그룹의 비율을 20%에서 50%까지 5% 단위로 변경
BIAS_STEPS = np.arange(0.2, 0.55, 0.05) 

class FAIREBiasedSampler(Sampler):
    """FAIRE 데이터셋에서 특정 그룹의 비율을 조절하는 샘플러"""
    def __init__(self, dataset, target_group_name, desired_group_ratio):
        self.dataset = dataset
        self.indices = dataset.indices # Subset의 인덱스를 가져옴
        self.original_df = dataset.dataset.df # 원본 데이터프레임 참조
        
        # 목표 그룹과 나머지 그룹의 인덱스를 식별
        target_group_mask = (self.original_df.loc[self.indices, SENSITIVE_ATTR] == target_group_name)
        self.target_indices = self.original_df.loc[self.indices][target_group_mask].index.tolist()
        self.other_indices = self.original_df.loc[self.indices][~target_group_mask].index.tolist()
        
        # 원하는 비율에 맞춰 각 그룹에서 샘플링할 개수 계산
        self.num_target_samples = int(len(self.indices) * desired_group_ratio)
        self.num_other_samples = len(self.indices) - self.num_target_samples

    def __iter__(self):
        # 각 그룹에서 필요한 만큼 샘플링 (복원 추출)
        target_samples = np.random.choice(self.target_indices, self.num_target_samples, replace=True)
        other_samples = np.random.choice(self.other_indices, self.num_other_samples, replace=True)
        
        final_indices = np.concatenate([target_samples, other_samples])
        np.random.shuffle(final_indices)
        
        # 반환되는 인덱스는 Subset에 대한 상대적 위치여야 하므로, 매핑이 필요.
        # 여기서는 간단히 원본 데이터셋 인덱스를 그대로 사용하나, DataLoader는 Subset을 사용해야 함.
        # DataLoader는 Subset을 사용하되, sampler는 원본 데이터셋의 인덱스를 반환.
        # --> 이 접근 방식은 복잡하므로, 여기서는 평가용 데이터로더를 매번 새로 정의.
        return iter(final_indices.tolist())

    def __len__(self):
        return len(self.indices)

def run_faire_sensitivity_analysis(model, dataset):
    results_list = []
    model.eval()
    
    for ratio in tqdm(BIAS_STEPS, desc="Analyzing Bias Sensitivity"):
        sampler = FAIREBiasedSampler(dataset, BIAS_INJECTION_GROUP, ratio)
        # 중요: Sampler가 전체 데이터셋의 인덱스를 반환하므로, DataLoader의 dataset도 full_dataset이어야 함.
        dataloader = DataLoader(dataset.dataset, batch_size=BATCH_SIZE, sampler=sampler, num_workers=NUM_WORKERS)
        
        eval_results = evaluate_faire_fairness(model, dataloader)
        eval_results['bias_ratio'] = ratio
        results_list.append(eval_results)
        
    return pd.DataFrame(results_list)

# --- 분석 실행 ---
if 'baseline_model' in locals():
    print("Starting Micro-Bias Sensitivity Analysis for FAIRE...")
    sensitivity_results_df = run_faire_sensitivity_analysis(baseline_model, val_dataset)
    print("Analysis finished.")

    # --- 결과 시각화 ---
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7))
    fig.suptitle(f"FAIRE - Micro-Bias Sensitivity Curve (Injecting bias for '{BIAS_INJECTION_GROUP}' group)", fontsize=16)

    ax1.plot(sensitivity_results_df['bias_ratio'], sensitivity_results_df['overall_acc'], 
             marker='o', linestyle='-', label='Overall Accuracy', linewidth=2.5)
    ax1.set_xlabel(f"Proportion of '{BIAS_INJECTION_GROUP}' samples in dataset")
    ax1.set_ylabel('Accuracy'); ax1.set_title('Accuracy vs. Injected Bias')
    ax1.legend(); ax1.grid(True, which='both', linestyle='--')

    ax2.plot(sensitivity_results_df['bias_ratio'], sensitivity_results_df['accuracy_gap'], 
             marker='o', linestyle='-', color='r', label='Max Accuracy Gap')
    ax2.set_xlabel(f"Proportion of '{BIAS_INJECTION_GROUP}' samples in dataset")
    ax2.set_ylabel('Max Accuracy Gap (Higher is worse)'); ax2.set_title('Fairness Metric vs. Injected Bias')
    ax2.legend(); ax2.grid(True, which='both', linestyle='--')
    
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig(os.path.join(RESULTS_DIR, 'faire_micro_bias_sensitivity_curve.png'))
    plt.show()
else:
    print("Baseline model not available. Please run Step 2 first.")
    sensitivity_results_df = None


---
## 단계 4: Over-Correction Damage Index (ODI, 과보정 피해 지수) 분석

편향 완화 기법 적용의 효율성을 평가하기 위해 ODI를 계산합니다.

**수행 작업:**
1.  **편향 완화 기법 적용 (Reweighing)**: 학습 데이터셋에서 소수 인종 그룹에 더 높은 가중치를 부여하여 편향을 완화하는 `Reweighing` 기법을 적용합니다. 가중치가 적용된 `WeightedRandomSampler`를 사용해 새로운 RoBERTa 모델을 재학습합니다.
2.  **완화 모델 평가 및 ODI 계산**: 재학습된 모델의 성능을 평가하고, Baseline 모델의 결과와 비교하여 "편향 감소량 대비 성능 손실량"을 나타내는 ODI를 계산합니다.


In [ ]:
# === 4.1 Reweighing 기법 적용 및 ODI 계산 ===

if 'baseline_results' in locals():
    # --- 1. Reweighing을 위한 가중치 계산 ---
    print("--- Step 1: Calculating weights for Reweighing ---")
    
    # 학습 데이터셋의 원본 DataFrame 정보를 가져옵니다.
    train_df = train_dataset.dataset.df.iloc[train_dataset.indices]
    
    group_counts = train_df[SENSITIVE_ATTR].value_counts()
    num_samples = len(train_df)
    group_weights = {group: num_samples / (len(group_counts) * count) for group, count in group_counts.items()}

    print("Group counts in training data:\n", group_counts)
    print("\nCalculated group weights:\n", group_weights)
    
    sample_weights = train_df[SENSITIVE_ATTR].apply(lambda group: group_weights[group]).to_numpy()
    sampler = WeightedRandomSampler(torch.from_numpy(sample_weights).double(), len(sample_weights))
    
    reweighed_train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=sampler, num_workers=NUM_WORKERS)
    reweighed_dataloaders = {'train': reweighed_train_loader, 'val': dataloaders['val']}
    print("\nCreated a new train dataloader with WeightedRandomSampler.")

    # --- 2. Reweighing 모델 재학습 ---
    print("\n--- Step 2: Retraining model with Reweighing ---")
    reweighed_model = get_faire_model(num_classes=5)
    optimizer = AdamW(reweighed_model.parameters(), lr=LEARNING_RATE, correct_bias=False)
    total_steps = len(reweighed_dataloaders['train']) * NUM_EPOCHS
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
    
    reweighed_model, _ = train_model(reweighed_model, reweighed_dataloaders, optimizer, scheduler, num_epochs=NUM_EPOCHS)

    # --- 3. 완화 모델 평가 및 ODI 계산 ---
    print("\n--- Step 3: Evaluating mitigated model and calculating ODI ---")
    reweighed_results = evaluate_faire_fairness(reweighed_model, dataloaders['val'])

    print("\n--- Reweighed Model Evaluation Results ---")
    print(f"Overall Accuracy: {reweighed_results['overall_acc']:.4f}")
    print(f"Maximum Accuracy Gap: {reweighed_results.get('accuracy_gap', 0):.4f}")

    accuracy_drop = baseline_results['overall_acc'] - reweighed_results['overall_acc']
    fairness_gain = baseline_results['accuracy_gap'] - reweighed_results['accuracy_gap']

    odi = accuracy_drop / fairness_gain if fairness_gain > 1e-6 else float('inf')

    print("\n--- Over-Correction Damage Index (ODI) ---")
    print(f"Accuracy Drop: {accuracy_drop:.4f}")
    print(f"Fairness Gain (Gap Reduction): {fairness_gain:.4f}")
    print(f"ODI (Accuracy Drop / Fairness Gain): {odi:.4f}")
else:
    print("Baseline model results are not available. Please run the full pipeline from Step 2.")


---
## 단계 5: Hidden Subgroup Discovery (잠복 하위집단 탐지)

단일 속성(`race` 또는 `gender`) 분석에서는 드러나지 않는, 두 속성이 교차하는 특정 하위집단(예: '흑인 여성')에서의 잠재적 편향을 탐지합니다.

**수행 작업:**
1.  **전체 예측 결과 생성**: Baseline 모델을 사용하여 검증 데이터셋의 모든 샘플에 대한 예측값과 실제값, 그리고 민감 속성(`race`, `gender`)을 하나의 데이터프레임으로 통합합니다.
2.  **하위집단별 성능 분석**: `race`와 `gender`를 조합하여 생성된 모든 하위집단에 대해 정확도를 계산합니다.
3.  **결과 시각화**: 하위집단별 정확도를 막대그래프로 시각화하고, 전체 평균 정확도와 비교하여 성능이 가장 낮은 최악의 하위집단을 식별합니다.


In [ ]:
# === 5.1 잠복 하위집단 분석 실행 ===

def get_faire_predictions_for_subgroup_analysis(model, dataloader):
    """FAIRE 데이터셋 전체에 대한 예측을 수행하고 상세 결과 데이터프레임을 반환합니다."""
    model.eval()
    
    y_true, y_pred, genders, races = [], [], [], []
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Getting predictions for subgroup analysis"):
            input_ids = batch['input_ids'].to(DEVICE)
            attention_mask = batch['attention_mask'].to(DEVICE)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)
            
            y_true.extend(batch['targets'].cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            genders.extend(batch['gender'].cpu().numpy())
            races.extend(batch['race'].cpu().numpy())
    
    original_dataset = dataloader.dataset.dataset
    idx_to_gender = original_dataset.idx_to_gender
    idx_to_race = original_dataset.idx_to_race
    
    df = pd.DataFrame({
        'true_label': y_true,
        'predicted_label': y_pred,
        'gender': [idx_to_gender[i] for i in genders],
        'race': [idx_to_race[i] for i in races]
    })
    df['correct'] = (df['true_label'] == df['predicted_label'])
    return df

# --- 분석 실행 ---
if 'baseline_model' in locals():
    predictions_df = get_faire_predictions_for_subgroup_analysis(baseline_model, dataloaders['val'])
    
    subgroup_analysis = predictions_df.groupby(['race', 'gender']).agg(
        accuracy=('correct', 'mean'),
        count=('correct', 'size')
    ).reset_index().sort_values(by='accuracy', ascending=True)

    print("\n--- Hidden Subgroup Analysis Results (Race x Gender) ---")
    display(subgroup_analysis)

    # --- 결과 시각화 ---
    plt.figure(figsize=(15, 8))
    subgroup_analysis['subgroup'] = subgroup_analysis['race'] + '-' + subgroup_analysis['gender']
    bars = sns.barplot(x='accuracy', y='subgroup', data=subgroup_analysis, palette='plasma')
    plt.title('FAIRE - Hidden Subgroup Performance (Race x Gender)', fontsize=16)
    plt.xlabel('Accuracy'); plt.ylabel('Subgroup'); plt.xlim(0, 1.0)
    
    overall_acc = predictions_df['correct'].mean()
    plt.axvline(x=overall_acc, color='r', linestyle='--', label=f'Overall Accuracy ({overall_acc:.3f})')
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, 'faire_hidden_subgroup_analysis.png'))
    plt.show()

else:
    print("Baseline model not available. Please run the full pipeline from Step 2.")
